In [ ]:

!pip install spellchecker
!pip install pyspellchecker
#!pip install simpletransformers
#!pip install torch


## Testando limpeza/correção com manutenção dos indices
- Com SpaCy

In [1]:
from __future__ import annotations
from spacy import displacy, tokens
from ..utils import dataset
from ..utils.annotation import preprocess, category_filters, comparison


In [2]:
import spacy
spacy.__version__

'3.2.4'

In [3]:
import spacy
nlp = spacy.load('pt_core_news_md')
nlp.add_pipe('sentencizer')

In [4]:
remover = preprocess.TokenRemover()
corrector = preprocess.TokenCorrector()
hyperseg = preprocess.HypersegmentationCorrector()
hyposeg = preprocess.HyposegmentationCorrector()
#punct = preprocess.PuntuationRestorer()

In [5]:
def log(text: str):
    print(text.center(80, '-'))

def print_sents(doc):
    print(*doc.sents, sep=' 🔶 ')

def print_sent_starts(doc):
    for tok in doc:
        print(tok.is_sent_start, '\t-->', tok.text)

Exemplo para Debug

In [ ]:
text = '''
texto para OOV testar errus, contidos em anotassões
OOV de trechos Ontem eu fui para Recife.
Eu goto de uva
Quando a menina gos tava de tinta e Marcos fes uma sur presa
Ontem eufoi pra casa.
'''
doc = nlp(text)
doc.spans['errors'] = [
    tokens.Span(doc, 5, 6, 'ortografia'),
    tokens.Span(doc, 9, 10, 'ortografia'),
    tokens.Span(doc, 22, 23, 'ortografia'),
    tokens.Span(doc, 29, 31, 'hiperseg'),
    tokens.Span(doc, 35, 36, 'ortografia'),
    tokens.Span(doc, 37, 39, 'hiperseg'),
    tokens.Span(doc, 41, 42, 'hiposeg'),
]
displacy.render(doc, 'spans', options=dict(spans_key='errors'))

In [ ]:
print_sents(doc)

In [ ]:
displacy.render(doc, 'span', options=dict(spans_key='errors'))

In [ ]:


log('REMOVENDO "OOV" e "\\n"')
res = remover.remove(doc, {'OOV', '\n'}, add_space=True)
displacy.render(res, 'span', options=dict(spans_key='errors'))

log('CORRIGINDO TEXTO')
res = corrector.correction(res)
displacy.render(res, 'span', options=dict(spans_key='errors'))

log('CORRIGINDO HYPERSEGMENTAÇÕES')
res = hyperseg.correction(res)
displacy.render(res, 'span', options=dict(spans_key='errors'))

log('CORRIGINDO HIPOSEGMENTAÇÕES')
res = hyposeg.correction(res)
displacy.render(res, 'span', options=dict(spans_key='errors'))

# log('RESTAURANDO PONTUAÇÃO')
# final = punct(res)
# displacy.render(final, 'span', options=dict(spans_key='errors'))

In [ ]:
log('Original')
print(res)
log('Restaurador na Biblioteca')
#print(final)

## Processando Textos

In [ ]:
def remove_bad_tokens(doc):
    return remover.remove(doc, words={'OOV', '\n'}, when=lambda token: token.is_space, add_space=True)

In [7]:
docs = dataset.create('../../raw_datasets/', merge=False, span_key="annot1")
print(*docs[0].spans.keys())
print('Documentos:', len(docs))

annot1 annot2 narrative punctuation orthography plagiarism storyline semantic syntax stylistics morphology
Documentos: 454


In [8]:
docs = dataset.create('../../raw_datasets/', merge=False, span_key="annot2")
print(*docs[0].spans.keys())
print('Documentos:', len(docs))

annot1 annot2 narrative punctuation orthography plagiarism storyline semantic syntax stylistics morphology
Documentos: 454


In [ ]:
from tqdm import tqdm

In [ ]:
for i, doc in tqdm(enumerate(docs), total=len(docs)):
    docs[i] = remove_bad_tokens(doc)

In [ ]:
for i, doc in tqdm(enumerate(docs), total=len(docs)):
    docs[i] = corrector(doc)

In [ ]:

for i, doc in tqdm(enumerate(docs), total=len(docs)):
    docs[i] = hyperseg(doc)

# docs = process_map(hyperseg, docs, tqdm_class=tqdm, max_workers=4, chunksize=20, desc='CORRIGINDO HYPERSEGMENTAÇÕES', total=len(docs))

In [ ]:
for i, doc in tqdm(enumerate(docs), total=len(docs)):
    docs[i] = hyposeg(doc)

# docs = process_map(hyposeg, docs, tqdm_class=tqdm, max_workers=4, chunksize=20, desc='CORRIGINDO HIPOSEGMENTAÇÕES', total=len(docs))

In [ ]:
#docs = list(tqdm(map(punct, docs), desc='RESTAURANDO PONTUAÇÃO', total=len(docs)))

Salvando

In [ ]:
tokens.DocBin(docs=docs, store_user_data=True).to_disk('../data/apa-nlp-span-clean-fix-hyper-hypo.docbin')

## Separando sentenças

In [ ]:
from spacy import tokens
import spacy
nlp = spacy.load('pt_core_news_md')
nlp.add_pipe('sentencizer')

In [21]:
docs = list(tokens.DocBin().from_disk('../../raw_datasets/apa-nlp-span-clean-fix-hyper-hypo.docbin').get_docs(nlp.vocab))

In [22]:
def set_sentences(doc: tokens.Doc) -> tokens.Doc:
    for tok in doc[:-1]:
        tok.is_sent_start = doc[tok.i -1].text == '.'
    return doc

In [23]:
for doc in docs:
    doc = set_sentences(doc)
print_sents(docs[-1])

meu irmã mais novo tinha desarrumado meu quarto e b r ı e c o t ı nu ei ue i pe ce b ter o Brıguei com ele ecotınuei a fazer minha pintura quando acabei minha pintura peceb que a tanta era da minha irmã mais velha entao Pedi dinheiro para minha mãe e ela me deu quando fui na loja vi que a loJa esta fechada então resolvi fazer minha propria tanta e pedi a ajuda da minha vo pegue flores colorida da minha vo e no final minha irmã mais veja amo fim


In [24]:
docs[1].spans['punctuation'][0]

nesse dia

In [25]:
import math
def scale_number(unscaled, to_min, to_max, from_min, from_max):
    return math.floor((to_max-to_min)*(unscaled-from_min)/(from_max-from_min)+to_min)

In [26]:
import jsonlines
from collections import defaultdict
import pandas as pd
import re
special_pattern = re.compile(r'\s+|\n+|/n|\t+|-|—')
marks = re.compile(r'\[\w{0,3}|\W{0,3}\]|\(|\)')


In [27]:
def join_split_words(text):
    """
    Junta palavras separadas por um \n
    """

    text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)
    text = re.sub(r'(\w+)_\n(\w+)', r'\1\2', text)
    return text

def fix_break_lines(text):
    text = re.sub(r'/n', '\n', text)
    return text


def separate_punctuation(text):
    text = re.sub(r'([.,?!;:])(\w)', r'\1 \2', text)
    return text

def remove_before_sent(text):
    tokens = text.split()
    if len(tokens) > 0 and tokens[0] in [',', '.']:
        tokens.pop(0)

    return  ' '.join(tokens)
def join_punctuation_marks(text):
    text = re.sub(r'(\w)\s([.,?!;:]+)', r'\1\2', text)
    return text


def clean_text(text):
    """
    Remove caracteres especiais e espaços em branco e as
    marcações de início e fim de parágrafo e afins.
    :param text:
    :return:
    """
    text = re.sub(r'\[\?\}', '', text).strip()
    text = re.sub(special_pattern, ' ', text)
    text = re.sub(marks, '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\.+', '.', text)
    text = re.sub(r'\"', '', text).strip()
    text = re.sub(r'\[\?\}', '', text).strip()
    text = re.sub(r'[*+]', '', text)
    return ' '.join(text.split())

def replace_punct(text):
    text = re.sub(r'[!?;]', '.', text)
    text = re.sub(r'[:]', ',', text)
    return text

In [28]:
print(re.sub(r'\[.*\]', '', '[LT] olá mundo'))

 olá mundo


In [29]:
def preprocess_sentence(sent_):
    sent_ = re.sub(r'\[.*\]', "", sent_)
    sent_ = join_split_words(sent_)
    sent_ = fix_break_lines(sent_)
    sent_ = clean_text(sent_)
    sent_ = separate_punctuation(sent_)
    sent_ = join_punctuation_marks(sent_)
    sent_ = replace_punct(sent_)
    sent_ = remove_before_sent(sent_)
    return sent_.lower()

In [30]:


def get_ner_label( annot_):
    match_token = annot_.doc[annot.end-1].text
    condition_1 = (match_token != "." and annot_.label_ == "Erro de Pontuação")
    condition_3 = (match_token != "," and annot_.label_ == "Erro de vírgula")
    condition_4 = (match_token == "," and annot_.label_ == "Erro de vírgula")
    condition_5 = (match_token == "." and annot_.label_ == "Erro de Pontuação")
    if condition_1:
        label = "I-PERIOD"
    elif condition_3:
        label = "I-COMMA"
    elif condition_4 or condition_5:
        label = "O"
    else:
        raise ValueError(f"Condition for {match_token} and label {annot.label_} not implemented ")

    return label

annot_id = 0

results = []

ner_dataset = []
for text_id, doc in enumerate(docs):
    documento = defaultdict(set)
    spans = doc.spans['punctuation']

    ner_tags = ["O"] * len(doc)

    for i,token in enumerate(doc):
        if token.text in [".", ";", "!", "...", "?"]:
            ner_tags[i-1] = "I-PERIOD"
        elif token.text in [","]:
            ner_tags[i-1] = "I-COMMA"
   
    for  annot in comparison.merge(list(doc.sents), spans):

        if len(preprocess_sentence(annot.sent.text).split()) < 3:
            continue
        if annot.label !=0:
            ner_tags[annot.end -1 ] = get_ner_label(annot)
        documento[annot.sent].add(annot)
        sentence = preprocess_sentence(annot.sent.text)


    ner_dataset.append({
        'text_id':text_id,
        'text': preprocess_sentence(doc.text),
        'labels': ner_tags,
        'tokens': [token.text.lower() for token in doc],
        'tag': "both_annotators"
    })
    results.append(documento)

In [31]:
len(results)

454

In [32]:
import json

json.dump(ner_dataset, open('./both_annotators.json', 'w'), indent=4)

In [34]:
with jsonlines.open("./ner_dataset.jsonl", "w") as ner_writer:
    for data in ner_dataset:
        ner_writer.write(data)

In [ ]:
with jsonlines.open('dataset/annotations-explainable.jsonl', mode='w') as annotations:
    df_input = []
    annot_id = 0
    for text_id, dic in enumerate(results):
        for (sent, annot) in dic.items():
            values = set(a.label_ for a in annot)

            has_punctuation_error = category_filters.Pontuação.ERRO in values
            has_comma_error = 'Erro de vírgula' in values

            label = "pontuação correta"
            if has_punctuation_error and not has_comma_error:
                label = "error de pontuação"
            elif has_comma_error and not has_punctuation_error:
                label = "error de vírgula"
            elif has_punctuation_error and has_punctuation_error:
                label = "error de pontuação e vírgula"

            sentence = preprocess_sentence(sent.text)
            annot_text = [preprocess_sentence(a.text).lower() for a in annot]

            v = dict(
                text_id=text_id,
                annot_id=annot_id,
                text=sentence,
                label=label,
                period=has_punctuation_error,
                comma=has_comma_error
            )
            df_input.append(v)

            annotations.write({
                            'text_id':text_id,
                            'annot_id':annot_id,
                            'text': sentence,
                            'label': label,
                            'annotation_span': annot_text
                        })
            annot_id += 1
    df = pd.DataFrame(df_input)

In [ ]:
plot1 = df[df['comma'] & ~df['period']].loc[:, ["period", "comma"]].value_counts()
plot1

In [ ]:
df[df['period'] & ~df['comma']].count()

In [ ]:
df[df['period'] & df['comma']].count()

In [ ]:
df[~(df['period'] | df['comma'])]

In [ ]:
df[~(df['period'] | df['comma'])].count()

In [ ]:
import jsonlines
with jsonlines.open('annotations.jsonl', mode='w') as writer:

    for doc in tqdm(docs, total=len(docs)):
         tokens = []
         labels_ids = []
         labels = []
         for i,token in enumerate(doc):

             if i < len(doc)-1 and token.text == '.' and doc[i + 1].text == '.':
                    continue
             tokens.append(token.text)
             labels_ids.append(0)
             labels.append("O")
             span_end = [spans.end-1 for spans in doc.spans['punctuation'] if spans.end-1 < len(tokens)]

             for span in span_end:

                 labels[span] = "I-Punctuation Mistake"
                 labels_ids[span] = 1

             if token.text == '.':

                writer.write({
                    'id':i,
                    'text':doc.text,
                     "tokens": tokens,
                    'labels': labels,
                    'labels_ids': labels_ids
                })